In [1]:
import requests
from bs4 import BeautifulSoup
from konlpy.tag import Okt; okt = Okt()
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

C:\Users\andrew\.conda\envs\text3.5\lib\site-packages\smart_open\ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')
C:\Users\andrew\.conda\envs\text3.5\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
urls = ['https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=101&oid=014&aid=0004240408', 
       'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=101&oid=008&aid=0004229219',
       'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=001&aid=0010871291', 
       'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=103&oid=029&aid=0002531788']

In [3]:
documents = []
for idx, url in enumerate(urls):
    req = requests.get(url)
    soup = BeautifulSoup(req.content, 'html.parser')
    news = soup.find('div', id='articleBodyContents')
    nouns_ = okt.nouns(news.text)
    doc_ = []
    for i in nouns_:
        if len(i) > 1:
            doc_.append(i)
    documents.append(
        TaggedDocument(words = doc_, 
                       tags=['news_{}'.format(idx)]))

In [4]:
model = Doc2Vec(documents, 
                vector_size=200, 
                window=12, 
                min_count=5, 
                workers=4, 
                alpha=0.025, 
                min_alpha=0.025)

In [5]:
model.most_similar('중국', topn=8)

C:\Users\andrew\.conda\envs\text3.5\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('미국', 0.2656923830509186),
 ('지난해', 0.1564527451992035),
 ('분쟁', 0.1343425065279007),
 ('기자', 0.10465133190155029),
 ('금융투자', 0.10402713716030121),
 ('연합뉴스', 0.09884481877088547),
 ('수소', 0.0917060449719429),
 ('버스', 0.09019621461629868)]

In [6]:
model.most_similar('에너지', topn=8)

C:\Users\andrew\.conda\envs\text3.5\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('신형', 0.21298155188560486),
 ('충전', 0.18025653064250946),
 ('자동차', 0.14278078079223633),
 ('지난해', 0.13110071420669556),
 ('대통령', 0.12444975972175598),
 ('현대차', 0.11405081301927567),
 ('도심', 0.10032868385314941),
 ('수소', 0.09897743910551071)]

In [12]:
model.docvecs.count

4

In [11]:
model.docvecs.doctags

{'news_0': Doctag(offset=0, word_count=203, doc_count=1),
 'news_1': Doctag(offset=1, word_count=211, doc_count=1),
 'news_2': Doctag(offset=2, word_count=251, doc_count=1),
 'news_3': Doctag(offset=3, word_count=373, doc_count=1)}

In [7]:
len(model.docvecs)

4

In [8]:
for idx, doctag in sorted(model.docvecs.doctags.items(), key=lambda x:x[1].offset):
    print(idx, doctag)

news_0 Doctag(offset=0, word_count=203, doc_count=1)
news_1 Doctag(offset=1, word_count=211, doc_count=1)
news_2 Doctag(offset=2, word_count=251, doc_count=1)
news_3 Doctag(offset=3, word_count=373, doc_count=1)


In [9]:
model.docvecs.most_similar('news_0')

[('news_1', 0.2968999445438385),
 ('news_3', 0.24522429704666138),
 ('news_2', 0.14569976925849915)]

In [10]:
model.docvecs.most_similar(1)

[('news_0', 0.2968999445438385),
 ('news_2', 0.2092197835445404),
 ('news_3', 0.14126718044281006)]